# Trabajo Práctico N°2

In [2]:
%%writefile README.md

ALLENDE, Lucila Abigail
SERRUYA ALOISI, Luciano Sebastián
TOLEDO MARGALEF, Pablo Adrián

Writing README.md


In [144]:
%%writefile ej1.c
#include<stdio.h>
#include<stdlib.h>
#include<unistd.h>
#include<pthread.h>

#define MAX 3
#define ESPERA 500

pthread_t hilos[MAX];
pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t cond= PTHREAD_COND_INITIALIZER;

int contadorSegundoHilo = 0;


int esperaAleatoria(){
    return (((rand()%4)+1) * 100) * 1000;
}

void *tercerHilo(void *arg){
    
    
    int i;

    for(i = 1; i <= 100; i++){
        
        if (!(i%10)){
            /*
                Si el contador de este hilo llego a diez
                despierta al segundo hilo 
            */
            pthread_cond_signal(&cond);
        }
        
        printf(" %d %d\n", contadorSegundoHilo, i%10);
        usleep(esperaAleatoria());
    }
    
   pthread_exit((void *)0);
}

void *segundoHilo(void *contador){

    pthread_create(&hilos[2], NULL, &tercerHilo, NULL);
    while(contadorSegundoHilo<10){
        
        pthread_mutex_lock(&mutex);

        //esperamos contador interno del tercer hilo
        pthread_cond_wait(&cond,&mutex);
        //incrementamos
        contadorSegundoHilo++;

        pthread_mutex_unlock(&mutex);
        
    }
    
    pthread_exit((void *)0);
    
}

void *primerHilo(void *arg){
    int retval = pthread_create(&hilos[1], NULL, &segundoHilo,NULL);
    if(retval != 0)
        exit(1);
    
    //nos quedamos esperando al segundo hilo
    while(contadorSegundoHilo < 10);
    
    pthread_exit((void *)0);
}





int main(void){
    
    //generacion de random seed
    srand(getpid());
    
    //primer hilo
    int retval = pthread_create(&hilos[0], NULL, &primerHilo,NULL);
    if(retval != 0)
        exit(1);
    
    //joineamos al primer hilo
    pthread_join(hilos[0],NULL);
    
    return 0;
}


Overwriting ej1.c


In [145]:
!gcc -pthread ej1.c -o ej1


In [146]:
!./ej1

 0 1
 0 2
 0 3
 0 4
 0 5
 0 6
 0 7
 0 8
 0 9
 0 0
 1 1
 1 2
 1 3
 1 4
 1 5
 1 6
 1 7
 1 8
 1 9
 1 0
 2 1
^C


In [1]:
%%writefile ej2-mutex.c
#include<stdio.h>
#include<stdlib.h>
#include<unistd.h>
#include<pthread.h>

#define MAX 10
#define ESPERA 500

pthread_t hilos[MAX];
pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t cond= PTHREAD_COND_INITIALIZER;


void *meReBloqueo(void *arg){
    int i = (int)arg;
    printf("trato bloquear, hilo %d\n",i);
    pthread_mutex_lock(&mutex);
    //al desbloquearse el mutex en el main, este hilo se despierta
    printf("Me desperté, hilo %d\n", i);
    pthread_mutex_unlock(&mutex);
    
}


int main(void){
    pthread_mutex_lock(&mutex);
    
    int i; 
    for(i = 0; i < MAX; i++){
        //creamos hilos
        int retval = pthread_create(&hilos[i], NULL, &meReBloqueo,(void *)i);
        if(retval != 0)
            exit(1);     
    }
    
    //"despertamos" todos los hilos
    pthread_mutex_unlock(&mutex);
    
    for(i = 0; i < MAX; i++){
        //joineamos todos los hilos
        pthread_join(hilos[i],NULL);    
    }
    return 0;
}


Overwriting ej2-mutex.c


In [4]:
!gcc -pthread ej2-mutex.c -o ej2-mutex

In [5]:
!./ej2-mutex

trato bloquear, hilo 2
trato bloquear, hilo 1
trato bloquear, hilo 0
trato bloquear, hilo 3
trato bloquear, hilo 4
trato bloquear, hilo 5
trato bloquear, hilo 6
trato bloquear, hilo 7
trato bloquear, hilo 8
trato bloquear, hilo 9
Me desperté, hilo 2
Me desperté, hilo 1
Me desperté, hilo 0
Me desperté, hilo 3
Me desperté, hilo 4
Me desperté, hilo 5
Me desperté, hilo 6
Me desperté, hilo 7
Me desperté, hilo 8
Me desperté, hilo 9


In [100]:
%%writefile ej2-condicion.c
#include<stdio.h>
#include<stdlib.h>
#include<unistd.h>
#include<pthread.h>

#define MAX 10
#define ESPERA 500

pthread_t hilos[MAX];
pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
pthread_cond_t cond= PTHREAD_COND_INITIALIZER;


void *esperoCondicion(void *arg){
    int i = (int)arg;
    pthread_mutex_lock(&mutex);
    
    //espera la condicion
    pthread_cond_wait(&cond,&mutex);
    
    
    printf("Me desperté, hilo %d\n", i);
    
    pthread_mutex_unlock(&mutex);
    pthread_exit((void *)0);
}


int main(void){
    
    int i; 
    for(i = 0; i < MAX; i++){
        //creamos hilos
        int retval = pthread_create(&hilos[i], NULL, &esperoCondicion,(void *)i);
        if(retval != 0)
            exit(1);     
    }
    
    sleep(1);
    
    
    pthread_cond_broadcast(&cond);
    
    for(i = 0; i < MAX; i++){
        //joineamos todos los hilos
        pthread_join(hilos[i],NULL);    
    }
    return 0;
}

Overwriting ej2-condicion.c


In [98]:
!gcc -pthread ej2-condicion.c -o ej2-condicion

In [99]:
!./ej2-condicion

Me desperté, hilo 0
Me desperté, hilo 1
Me desperté, hilo 2
Me desperté, hilo 3
Me desperté, hilo 9
Me desperté, hilo 8
Me desperté, hilo 7
Me desperté, hilo 6
Me desperté, hilo 5
Me desperté, hilo 4


In [98]:
%%writefile ej3.c
#include<stdio.h>
#include<pthread.h>
#include<unistd.h>
#include<stdlib.h>

#define MAX_PILA  1000
#define MAX_HILOS 10
#define ESPERA    50000

pthread_t hilosPush[MAX_HILOS];
pthread_t hilosPop[MAX_HILOS];

typedef struct {
    int tope;
    pthread_cond_t llena, vacia;
    pthread_mutex_t mutex;
    int contenido[MAX_PILA];
} pila_t;

pila_t pila  = {-1, PTHREAD_COND_INITIALIZER, PTHREAD_COND_INITIALIZER, PTHREAD_MUTEX_INITIALIZER};
    
int numeroAleatorio(){
    return rand() % 10;
}
void *push (void *arg){
    while(1){
        pthread_mutex_lock(&pila.mutex);
        if(pila.tope == (MAX_PILA-1)){
            pthread_cond_wait(&pila.llena,&pila.mutex);
            printf("PILA_LLENA: Espero condicion\n");
        }
        //el tope apunta al elemento visible de la pila
        pila.contenido[++pila.tope] = numeroAleatorio();
        
        pthread_cond_broadcast(&pila.vacia);
        
        pthread_mutex_unlock(&pila.mutex);
        
    }
    pthread_exit((void *)0);
}

void *pop (void *arg){
    while(1){
        pthread_mutex_lock(&pila.mutex);
        if(pila.tope == -1){
            pthread_cond_wait(&pila.vacia,&pila.mutex);
            printf("PILA_VACIA: Espero condicion\n");
        }
        //el tope apunta al elemento visible de la pila
        printf("%d\n", pila.contenido[pila.tope]);
        usleep(ESPERA);
        pila.tope--;
        
        pthread_cond_broadcast(&pila.llena);
        
        pthread_mutex_unlock(&pila.mutex);
        
    }
    //pop desde la pila
    //si se vacia, espera condicion
    
    pthread_exit((void *)0);
}
int main(void){
    
    int i;
    int retval;
    srand(getpid());
    
    
    //hilos pusheadores
    for(i = 0; i < MAX_HILOS ; i++){
        retval = pthread_create(&hilosPush[i],NULL,&push, NULL);
        if(retval != 0)
            exit(1);
    }
    
    sleep(1);
    
    //hilos popeadores
    for(i = 0; i < MAX_HILOS ; i++){
        retval = pthread_create(&hilosPop[i],NULL,&pop, NULL);
        if(retval != 0)
            exit(1);
    }
    
    
    //esperamos al primer hilo creado
    pthread_join(hilosPush[0], NULL);
    return 0;
}

Overwriting ej3.c


In [99]:
!gcc -pthread ej3.c -o ej3

In [66]:
!./ej3

Hilos pusheadores creados
2
Hilos popeadores creados
7
1
0
9
3
2
6
9
0
8
6
8
7
1
8
5
0
8
4
0
7
2
1
3
2
1
5
6
9
7
0
8
6
8
1
2
0
7
1
2
1
5
7
2
1
0
8
8
1
0
7
4
9
7
5
4
2
8
6
9
9
^C
